# Appendix B - Graphs with multiple connected components

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

## Generate an example figure

In [ ]:
cc1 = nx.connected_watts_strogatz_graph(10, 3, 0.8)
cc2 = nx.connected_watts_strogatz_graph(10, 3, 0.8)
cc3 = nx.connected_watts_strogatz_graph(10, 3, 0.8)
graph_demo = nx.disjoint_union(cc1, cc2)
graph_demo = nx.disjoint_union(graph_demo, cc3)

nx.draw_spring(graph_demo, node_color='g')
plt.savefig('../figures/multiple-component.eps')
plt.show()

## Plot eigenvalues

In [ ]:
n_graphs = 100
n_nodes = 64

for n_con_com in [2, 4, 8]: # Plot the boxplot for each type of graph
    eigenvalues_graphs = []
    
    for _ in range(n_graphs): # Compute the eigenvalues for each graph
        # Build the graph
        graph = nx.Graph()
        for _ in range(n_con_com): # Create the connected components
            cc = nx.connected_watts_strogatz_graph(n_nodes//n_con_com, 2, 0.2)
            graph = nx.disjoint_union(graph, cc)
            
        eigenvalues = nx.laplacian_spectrum(graph)
        eigenvalues_graphs.append(eigenvalues)
            
    eigenvalues_graphs = np.array(eigenvalues_graphs)
    
    # Assert validity of the assumption
    results = []
    for idx in range(n_con_com):
        eigenvalue = eigenvalues_graphs[:, idx]
        results.append(np.allclose(eigenvalue, np.zeros_like(eigenvalue)))
    print(f'Are the first {n_con_com} eigenvalues always zero?', np.all(results))
    
    results = []
    for idx in range(n_con_com, eigenvalues_graphs.shape[1]):
        eigenvalue = eigenvalues_graphs[:, idx]
        results.append(np.allclose(eigenvalue, np.zeros_like(eigenvalue)))
    print('Are the other eigenvalues always nonzero?', not np.any(results))

    # Boxplot
    plt.figure(n_con_com)
    plt.boxplot(eigenvalues_graphs[:,0:10], showfliers=False)
    plt.xticks(np.arange(1, 10+1))
    plt.xlabel('Eigenvalues')
    plt.ylabel('Intensity')
    plt.savefig(f'../figures/{n_con_com}-components.eps')
    plt.show()

## Plot eigenvectors with the relative eigenvalue equals to zero

In [ ]:
# Generate adjacency matrix
n_nodes = 10
W = np.zeros((n_nodes, n_nodes))
W[0:3, 0:3] = 1
W[3:8, 3:8] = 1
W[8:, 8:] = 1

# Compute the laplacian
D = np.diag(np.sum(W, axis=0))
laplacian = D - W

# Compute eigenvalues and eigenvectors
evals, evects = np.linalg.eigh(laplacian)

W

In [ ]:
# Plot eigenvalues
plt.bar(np.arange(n_nodes), evals)
plt.xticks(np.arange(0, n_nodes), labels=np.arange(1, n_nodes+1))
plt.xlabel('Eigenvalues')
plt.ylabel('Intensity')
plt.show()

In [ ]:
# Plot eigenvectors
for idx in range(3):
    plt.figure(idx)
    plt.bar(np.arange(n_nodes), evects[:,idx])
    plt.xticks(np.arange(0, n_nodes), labels=np.arange(1, n_nodes+1))
    plt.xlabel('Nodes')
    plt.ylabel('Intensity')
    plt.title(f'Eigenvector {idx+1}')
    plt.savefig(f'../figures/{idx}-eigenvectors.eps')
    plt.show()